In [1]:
import h5py
import napari
import zarr
from abbott.h5_files import *
from pathlib import Path
import imageio as io
from pathlib import Path
import pandas as pd
from skimage.filters import gaussian

In [56]:

fld = Path(r"Q:\carspi\ETiX all sample 384 overview")

In [74]:


imgs = {}
for fn in fld.glob("*.h5"):

    with h5py.File(fn) as f:
         
        bf=f['ch_00/2'][...]
        #mask = f['/mask_cleanup_corrected/1'][...]
        #gfp = f['ch_01/1'][...]
        #mcherry = f['ch_02/1'][...]

        
        site = fn.stem
        imgs[site] = bf


In [19]:
imgs[site] =bf

In [ ]:
#for bry signal ?

bry=mcherry-gfp

difference = gaussian(difference, sigma=2)
binary = difference < 0.6


In [75]:
# arrange embryos on 384 well grid

import re

pattern = re.compile(r'\d{1,3}')
sorted_imgs = sorted(imgs.keys(), key=lambda x: int("".join(pattern.findall(x.split("_")[1]))))

order = pd.Series(data= np.arange(len(imgs.keys())), index= sorted_imgs)

In [21]:
#only one timepoint

from functools import partial
from typing import Union, Any
import numpy as np
from tqdm import tqdm
from numpy.typing import NDArray
from itertools import product, repeat
from typing import Callable
import pandas as pd


def rect_grid_384(sites: pd.Series, left_to_right=True, aspect_ratio=2) -> pd.DataFrame:
    n = len(sites)
    ny = 24
    nx = 16
    indices = np.array(list(product(np.arange(ny), np.arange(nx))))
    if left_to_right:
        indices = np.flip(indices, axis=1)
    return pd.DataFrame(indices[:len(sites)], columns=['iy', 'ix'], index=sites.sort_values().index)

def arrange_on_grid(raw_imgs: dict[str, NDArray],
                    order: pd.Series,
                    index_function: Callable[[pd.Series], pd.DataFrame] = rect_grid_384,
                    margin_px: int = 10) -> NDArray:
    embs = list(set(raw_imgs.keys()).intersection(set(order.index)))
    order = order.loc[embs]
    imgs = {k: v for k, v in raw_imgs.items() if k in embs}

    dtype = imgs[list(imgs.keys())[0]].dtype
    max_extent = np.array([img.shape for img in imgs.values()]).max(axis=0)

    dy_site, dx_site = max_extent
    dy_margin = dx_margin = margin_px

    site_indices = index_function(order)

    canvas_extent_yx = (site_indices.max().values + 1) * np.array(
        [dy_site + dy_margin, dx_site + dx_margin]) - np.array([dy_margin, dx_margin])

    canvas = np.zeros(canvas_extent_yx, dtype=dtype)
    for site, index in site_indices.iterrows():
        dy_current, dx_current = imgs[site].shape
        dy, dx = index.values * np.array([dy_site + dy_margin, dx_site + dx_margin])
        canvas[dy:dy + dy_current, dx:dx + dx_current] = imgs[site]

    return canvas



In [76]:
#timeline grid
from functools import partial
from typing import Union, Any
import numpy as np
from tqdm import tqdm
from numpy.typing import NDArray
from itertools import product, repeat
from typing import Callable
import pandas as pd


def rect_grid_384(sites: pd.Series, left_to_right=True, aspect_ratio=2) -> pd.DataFrame:
    n = len(sites)
    ny = 24
    nx = 16
    indices = np.array(list(product(np.arange(ny), np.arange(nx))))
    if left_to_right:
        indices = np.flip(indices, axis=1)
    return pd.DataFrame(indices[:len(sites)], columns=['iy', 'ix'], index=sites.sort_values().index)

def arrange_on_grid(raw_imgs: dict[str, NDArray],
                    order: pd.Series,
                    index_function: Callable[[pd.Series], pd.DataFrame] = rect_grid_384,
                    margin_px: int = 10) -> NDArray:
    embs = list(set(raw_imgs.keys()).intersection(set(order.index)))
    order = order.loc[embs]
    imgs = {k: v for k, v in raw_imgs.items() if k in embs}

    dtype = imgs[list(imgs.keys())[0]].dtype
    max_extent = np.array([img.shape for img in imgs.values()]).max(axis=0)

    dz_site, dy_site, dx_site = max_extent
    dy_margin = dx_margin = margin_px

    site_indices = index_function(order)

    canvas_extent_yx = (site_indices.max().values + 1) * np.array(
        [dy_site + dy_margin, dx_site + dx_margin]) - np.array([dy_margin, dx_margin])

    canvas = np.zeros((dz_site, *canvas_extent_yx), dtype=dtype)
    for site, index in site_indices.iterrows():
        _, dy_current, dx_current = imgs[site].shape
        dy, dx = index.values * np.array([dy_site + dy_margin, dx_site + dx_margin])
        canvas[:, dy:dy + dy_current, dx:dx + dx_current] = imgs[site]

    return canvas

In [77]:
canvas = arrange_on_grid(imgs, order)

viewer = napari.Viewer()
#viewer.add_labels(canvas)
viewer.add_image(canvas)

<Image layer 'canvas' at 0x2099a217130>